# Otimização de Portfólio de Investimentos

**Disciplina** : Otimização - 2025.2

**Professor** : Luidi Simonetti

**Alunos** : Bernardo Pozzato e Pedro Cintra

---

## 1. Introdução:
### 1.1 Motivação:

O tema escolhido para o nosso trabalho se deu por conta do nosso interesse crescente pelo mercado financeiro e pela complexidade inerente à tomada de decisão em investimentos. Em um cenário econômico globalizado, investidores deparam-se com uma vastidão de ativos disponíveis — desde ações tradicionais e títulos públicos até novos instrumentos como criptoativos.

A seleção manual de uma carteira de investimentos é frequentemente ineficiente, pois é difícil para um ser humano quantificar intuitivamente como o risco de um ativo interage com o de outro. A simples diversificação ingênua (dividir o capital igualmente) pode não proteger o patrimônio contra riscos sistêmicos ou correlações ocultas. Portanto, a motivação deste projeto reside na aplicação de métodos quantitativos e computacionais para resolver esse dilema, trazendo rigor matemático para maximizar a eficiência da alocação de capital. Buscamos entender como algoritmos de otimização podem superar a intuição humana na construção de portfólios robustos.

### 1.2 Definição do problema:

O nosso problema consiste em, com base numa gama de ativos pré-selecionados (universo de investimento), definir e encontrar a carteira ótima: aquela que apresenta o maior retorno esperado possível para um nível de risco aceitável, ou, dualmente, o menor risco possível para uma meta de retorno.

Matematicamente, tratamos isso como um problema de otimização multiobjetivo (Risco vs. Retorno) que é scalarizado para uma função objetivo única. A construção da solução depende fundamentalmente de parâmetros (inputs) fornecidos pelo perfil do investidor:

* **Fator Lambda ($\lambda$)**: Este parâmetro é a representação numérica da Aversão ao Risco do usuário. Ele atua como um "peso" na função objetivo, definindo quanto de retorno o investidor exige para aceitar uma unidade adicional de risco (variância).

* Lambda Alto (ex: 50): O algoritmo priorizará agressivamente a redução da variância. O resultado tende a ser uma carteira conservadora (focada em Renda Fixa ou utilidades).

* Lambda Baixo (ex: 0.1): O algoritmo tolerará alta volatilidade em busca de retornos marginais maiores.

* **Teto de Risco ($\sigma_{max}$)**: Diferente do Lambda (que está na função objetivo), este é um parâmetro de restrição rígida (constraint). Ele define o limite superior de volatilidade (desvio padrão anual) que a carteira pode ter. Por exemplo, se o usuário define um teto de 15%, qualquer combinação de ativos que resulte em uma volatilidade de 15,1% será descartada como "inviável", independentemente de quão alto seja seu retorno.

Para operacionalizar isso em código, primeiramente definimos nosso Universo de Ativos e os parâmetros de entrada no arquivo `config.py`. Isso delimita o escopo da nossa otimização.


```python
# Trecho de: config.py
# --- PARÂMETROS GERAIS ---
ANOS_DE_DADOS = 5
LAMBDA_AVERSAO_RISCO_PADRAO = 2.0 
# --- UNIVERSO DE ATIVOS (Excerto) ---
UNIVERSO_ATIVOS = {
    "SETOR_ENERGIA_PETROLEO": [
        'PETR3.SA', 'PETR4.SA', 'PRIO3.SA', 'CSAN3.SA', ...
    ],
    "SETOR_FINANCEIRO_SEGUROS": [
        'ITUB4.SA', 'BBDC4.SA', 'BBAS3.SA', 'SANB11.SA', ...
    ],
    "CRIPTOATIVOS": [
        'BTC-USD', 'ETH-USD', 'SOL-USD', ...
    ],
    "RENDA_FIXA_SIMULADA": [
        'ATIVO_SELIC_SIMULADO' # Ativo sintético para caixa
    ]
}
```

### 1.3 Contexto Teórico e Abordagem

Historicamente, a Teoria Moderna de Portfólio (MPT), proposta por Harry Markowitz em 1952, estabeleceu as bases para resolver esse problema. Markowitz demonstrou que o risco de uma carteira não é apenas a soma ponderada dos riscos individuais, mas depende crucialmente da covariância entre os ativos.

Para resolver este problema computacionalmente, adotamos duas abordagens complementares:

* **Meta-heurística (Algoritmo Genético)**: Utilizamos a biblioteca pymoo para explorar o espaço de busca. Esta abordagem é especialmente útil para lidar com restrições não-lineares e descontinuidades, como a "Restrição de Setores" (onde o peso de um setor deve ser exatamente zero se proibido), que podem dificultar métodos baseados em gradiente.

* **Método Exato (Solver Gurobi)**: Empregamos programação quadrática para validar a convergência da heurística. O Gurobi nos fornece a solução matematicamente ótima (Global Optimum), servindo de benchmark para avaliarmos a qualidade da solução encontrada pelo Algoritmo Genético.

O sistema foi desenvolvido inteiramente em Python, integrando coleta de dados reais (Yahoo Finance para ações, API do Banco Central para Taxa Selic) e visualização automática dos resultados.

## 2. Modelo Matemático:

O cerne deste trabalho baseia-se na formulação de Média-Variância (Mean-Variance Framework). Neste modelo, assumimos que o investidor é um agente racional que deseja maximizar sua utilidade, a qual é positivamente correlacionada com o retorno esperado e negativamente correlacionada com a variância dos retornos (risco).

### 2.1 Variáveis e Parâmetros

Para a construção do modelo, definimos as seguintes notações:

- $N$: Número total de ativos no universo de investimento.

- $w$: Vetor de decisão ($N \times 1$), onde $w_i$ representa a proporção do capital total alocado no ativo $i$.

- $\mu$: Vetor de retornos esperados ($N \times 1$), calculado a partir da média histórica dos retornos diários anualizados.

- $\Sigma$: Matriz de Covariância ($N \times N$), onde cada elemento $\sigma_{ij}$ representa a covariância entre os retornos do ativo $i$ e do ativo $j$.

- $\lambda$: Coeficiente de aversão ao risco (escalar $\ge 0$).

A Matriz de Covariância é o componente fundamental do modelo. A variância total do portfólio ($\sigma_p^2$) é dada pela forma quadrática:

$$\sigma_p^2 = w^T \Sigma w = \sum_{i=1}^{N}\sum_{j=1}^{N} w_i w_j \sigma_{ij}$$

Essa equação demonstra matematicamente o benefício da diversificação: se selecionarmos ativos com covariância negativa ou baixa ($\sigma_{ij} < 0$), o termo cruzado reduz a variância total do portfólio, permitindo a redução de risco sem necessariamente reduzir o retorno.

No código `(preparar_dados.py)`, calculamos esses parâmetros a partir de dados históricos baixados do Yahoo Finance. Note que anualizamos os valores multiplicando por 252 (dias úteis).

```python

# Trecho de: preparar_dados.py
def calcular_inputs_otimizacao(valor_total_investido):
    # ... (Download dos dados omitido para brevidade) ...
    
    # Cálculo dos Retornos Diários
    retornos = precos.pct_change().dropna()
    
    # 1. Vetor de Retornos Esperados (Mi) - Anualizado
    retornos_medios_anuais = retornos.mean() * 252
    
    # 2. Matriz de Covariância (Sigma) - Anualizada
    matriz_cov_anual = retornos.cov() * 252
    
    return {
        'retornos_medios': retornos_medios_anuais,
        'matriz_cov': matriz_cov_anual,
        # ...
    }
```
### 2.2 Função Objetivo

O problema de otimização multiobjetivo (Maximizar Retorno, Minimizar Risco) é convertido em um problema mono-objetivo através da scalarização linear. Em nosso código, optamos pela minimização da "Desutilidade":

$$\text{Minimizar } f(w) = \lambda \underbrace{(w^T \Sigma w)}_{\text{Risco (Variância)}} - \underbrace{(w^T \mu)}_{\text{Retorno Esperado}}$$

Essa equação é implementada diretamente no método `_evaluate` da nossa classe de problema no `pymoo`. O uso de `np.einsum` garante eficiência computacional para calcular a variância de múltiplos indivíduos da população simultaneamente.

```python

# Trecho de: modelo_problema.py
def _evaluate(self, x, out, *args, **kwargs):
    # x: Matriz (População x Ativos) contendo os pesos (w)
    
    # Termo 1: Retorno (w * mu)
    retorno_calculado = x.dot(self.retornos_medios)
    
    # Termo 2: Variância (w * Sigma * w^T)
    # np.einsum é usado para multiplicação eficiente de matrizes em lote
    variancia = np.einsum('...i,ij,...j->...', x, self.matriz_cov, x)
    
    # FUNÇÃO OBJETIVO: Min (Lambda * Variância - Retorno)
    funcao_objetivo = (self.lambda_aversao_risco * variancia) - retorno_calculado
    
    out["F"] = funcao_objetivo
```

### 2.3 Restrições do Problema

O espaço de soluções viáveis é delimitado por restrições financeiras e operacionais rigorosas:

1. **Restrição de Orçamento ($\sum w_i = 1$) e Não-Negatividade ($w_i \ge 0$)**:
No Algoritmo Genético, garantimos isso através de um Operador de Reparo. Isso assegura que, mesmo após cruzamentos e mutações, o indivíduo resultante seja uma carteira válida (soma 100%).


$$\sum_{i=1}^{N} w_i = 1$$

```python
    # Trecho de: otimizar.py
    class MinimumWeightRepair(Repair):
        def _do(self, problem, X, **kwargs):
            # Remove pesos insignificantes (limpeza de ruído)
            X[X < 0.005] = 0.0
            
            # Garante soma = 1.0 (Normalização)
            somas = X.sum(axis=1, keepdims=True)
            somas[somas == 0] = 1.0 
            X = X / somas
            
            return X
```


2. **Restrição de Teto de Risco ($\sqrt{w^T \Sigma w} \le \sigma_{max}$)**: Esta é uma restrição de desigualdade tratada pelo `pymoo`.

```python
    # Trecho de: modelo_problema.py
        # Cálculo do Desvio Padrão
        risco_calculado = np.sqrt(np.maximum(variancia, 1e-12))
        # Restrição: Risco Calculado - Teto <= 0
        out["G"] = risco_calculado - self.risco_maximo_usuario
```

3. **Restrições Setoriais ($w_i = 0$ para proibidos)**:
Implementada alterando os limites superiores (xu) das variáveis de decisão. Se um ativo é proibido, seu peso máximo é forçado a 0.

```python
    # Trecho de: modelo_problema_setor.py
    class OtimizacaoPortfolioSetor(Problem):
        def __init__(self, ..., setores_proibidos):
            # ...
            xu = np.full(n_ativos, 1.0) # Limite padrão

            # Trava limite superior em 0 para ativos proibidos
            for setor in setores_proibidos:
                for ativo in mapa_setores[setor]:
                    idx = ticker_to_idx[ativo]
                    xu[idx] = 0.0 

            super().__init__(..., xu=xu)
```


## 3. Solução:



## 4. Discussão: